In [ ]:
# 1. Bersihkan environment lama
!pip uninstall -y pyarrow datasets evaluate transformers

# 2. Install versi STABIL & KOMPATIBEL (Sesuai Request)
!pip install pyarrow==14.0.1 datasets==2.15.0 evaluate==0.4.1 sacrebleu sentencepiece accelerate openpyxl -q
!pip install transformers==4.46.3 tokenizers==0.20.3 -q
!pip install pandas numpy scikit-learn -q



In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import torch
import gc
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
from transformers.optimization import Adafactor

# --- KONFIGURASI SESUAI SPEK SUKSES ---
# Model Pengganti Cendol: mt5-small (Ringan & Stabil)
MODEL_CHECKPOINT = "google/mt5-small" 

MAX_LENGTH = 128      # 100-128 aman
BATCH_SIZE = 4        # Batch Kecil biar aman
GRAD_ACCUM = 4        # Akumulasi 4x (Total Batch = 16)
EPOCHS = 5
OUTPUT_DIR = "/kaggle/working/model_madura_bolak_balik" 

# Cek GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# --- 1. LOAD DATASET (NusaX, INMAD, Lexicon) ---
data_total = []

# Path File (Sesuai request sebelumnya)
DIR_NUSAX   = "/kaggle/input/nusaxdata"
FILE_LEXICON = "/kaggle/input/nusaxdata/madurese.csv"
FILE_INMAD  = "/kaggle/input/inmad-dataset/INMAD Dataset.csv"

print("📂 Sedang membaca data...")

# A. Load NusaX
for f in glob.glob(f"{DIR_NUSAX}/*.csv"):
    if "madurese.csv" not in f and any(k in f for k in ['train', 'valid', 'test']):
        try:
            df = pd.read_csv(f)
            cols = [c.lower() for c in df.columns]
            temp = pd.DataFrame()
            if 'indonesian' in cols: temp['indo'] = df['indonesian']
            elif 'indonesia' in cols: temp['indo'] = df['indonesia']
            if 'madurese' in cols: temp['madura'] = df['madurese']
            elif 'madura' in cols: temp['madura'] = df['madura']
            
            if not temp.empty: data_total.append(temp)
        except: pass

# B. Load Lexicon
try:
    if os.path.exists(FILE_LEXICON):
        df_lex = pd.read_csv(FILE_LEXICON)
        temp_lex = pd.DataFrame({'indo': df_lex['indonesian'], 'madura': df_lex['madurese']})
        data_total.append(temp_lex)
except: pass

# C. Load INMAD
try:
    if os.path.exists(FILE_INMAD):
        df_inmad = pd.read_csv(FILE_INMAD)
        df_inmad.columns = [c.strip() for c in df_inmad.columns] # Hapus spasi nama kolom
        temp_inmad = pd.DataFrame({'indo': df_inmad['Indonesia'], 'madura': df_inmad['Madura']})
        data_total.append(temp_inmad)
except: pass

# Gabung Semua
df_raw = pd.concat(data_total, ignore_index=True)

# Bersihkan Data (Hapus yang kosong/pendek)
df_raw['indo'] = df_raw['indo'].astype(str).str.strip()
df_raw['madura'] = df_raw['madura'].astype(str).str.strip()
df_raw = df_raw[(df_raw['indo'].str.len() > 2) & (df_raw['madura'].str.len() > 2)]
df_raw = df_raw[(df_raw['indo'] != "nan") & (df_raw['madura'] != "nan")]

print(f"✅ Total Data Mentah: {len(df_raw)} pasang kalimat")

# --- 2. SETUP BOLAK-BALIK (MIRRORING) ---
# Arah 1: Indo -> Madura
df_indo_mad = pd.DataFrame({
    'source': df_raw['indo'],
    'target': df_raw['madura'],
    'prefix': "terjemahkan dari Bahasa Indonesia ke Bahasa Madura: "
})

# Arah 2: Madura -> Indo
df_mad_indo = pd.DataFrame({
    'source': df_raw['madura'],
    'target': df_raw['indo'],
    'prefix': "terjemahkan dari Bahasa Madura ke Bahasa Indonesia: "
})

# Gabung Jadi Satu Dataset Besar
df_final = pd.concat([df_indo_mad, df_mad_indo], ignore_index=True)
print(f"✅ Total Data Training (Bolak-Balik): {len(df_final)}")

# Split Data
train_df, val_df = train_test_split(df_final, test_size=0.1, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# --- 3. TOKENISASI ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def preprocess_function(examples):
    # Gabungkan Prefix + Source Text
    inputs = [p + str(s) for p, s in zip(examples["prefix"], examples["source"])]
    targets = [str(t) for t in examples["target"]]
    
    model_inputs = tokenizer(inputs, max_length=MAX_LENGTH, truncation=True)
    labels = tokenizer(text_target=targets, max_length=MAX_LENGTH, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("⏳ Sedang Tokenisasi...")
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)
print("✅ Tokenisasi Selesai.")

In [ ]:
import evaluate
import numpy as np

# Setup Metrics
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple): preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Ganti -100 di label agar valid
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

# 1. Reset Memori GPU
gc.collect()
torch.cuda.empty_cache()

# 2. Load Model
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
model.gradient_checkpointing_enable() # Hemat Memori
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 3. CONFIG TRAINING (PERSIS SPEK SUKSES)
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    
    # --- SPEK KUNCI ---
    per_device_train_batch_size=BATCH_SIZE, # 4
    per_device_eval_batch_size=BATCH_SIZE,  # 4
    gradient_accumulation_steps=GRAD_ACCUM, # 4 (Total Batch 16)
    gradient_checkpointing=True,
    optim="adafactor",           # Optimizer Wajib buat T5/MT5
    learning_rate=1e-3,          # LR standar Adafactor
    weight_decay=0.0,
    fp16=False,                  # WAJIB FALSE (Biar gak error NaN/0.00 loss)
    # ------------------
    
    save_total_limit=1,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    processing_class=tokenizer, 
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print(f"🚀 MULAI TRAINING MODEL: {MODEL_CHECKPOINT}")
trainer.train()

# Simpan Model Final
print("💾 Menyimpan model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ SELESAI!")

# eval

In [4]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
from tqdm import tqdm
import re

# ==========================================
# ⚙️ KONFIGURASI
# ==========================================
# 1. Path Model (Sesuai yang tadi)
MODEL_PATH = r"D:/UNAIR/NLP/Project_Madura/model_google_mt5"

# 2. Path File Test (Cari file test.csv dari NusaX di laptopmu)
# Kalau tidak ada, pakai file train.csv tapi kita ambil 100 baris aja buat tes
FILE_TEST_PATH = r"D:/UNAIR/NLP/Project_Madura/dataset4/test.csv"  # <--- GANTI INI DENGAN LOKASI FILE CSV KAMU!

# ==========================================
# 🔧 FUNGSI BERSIH-BERSIH (PENTING!)
# ==========================================
def bersihkan_teks(text):
    # Buang link https://...
    text = re.sub(r'http\S+', '', text)
    # Buang mention @user
    text = re.sub(r'@\w+', '', text)
    # Hapus spasi berlebih
    return text.strip()

# ==========================================
# 🚀 EVALUASI FULL
# ==========================================
def main():
    print(f"📂 Memuat model...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
        model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(device)
    except:
        print("❌ Model tidak ketemu. Cek path lagi.")
        return

    # Load Data Test
    print(f"📂 Membaca data tes: {FILE_TEST_PATH}")
    try:
        # Coba baca CSV. Sesuaikan nama kolomnya nanti
        df = pd.read_csv(FILE_TEST_PATH)
        
        # Cari kolom indo dan madura
        cols = [c.lower() for c in df.columns]
        src_col, tgt_col = None, None
        
        if 'indonesian' in cols: src_col = 'indonesian'
        elif 'indonesia' in cols: src_col = 'indonesia'
        
        if 'madurese' in cols: tgt_col = 'madurese'
        elif 'madura' in cols: tgt_col = 'madura'
        
        if not src_col or not tgt_col:
            print("❌ Kolom 'indonesian' atau 'madurese' tidak ditemukan di CSV.")
            print(f"Kolom yang ada: {cols}")
            return
            
        # Ambil 100 data saja biar cepat (kalau mau semua, hapus .head(100))
        df_sample = df.head(100) 
        print(f"✅ Menguji pada {len(df_sample)} kalimat pertama.")
        
    except Exception as e:
        print(f"❌ Gagal baca CSV: {e}")
        return

    metric = evaluate.load("sacrebleu")
    predictions = []
    references = []

    print("🚀 Mulai Menerjemahkan...")
    
    for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
        src_text = str(row[src_col])
        ref_text = str(row[tgt_col])
        
        # Tambah Prefix
        input_text = f"terjemahkan dari Bahasa Indonesia ke Bahasa Madura: {src_text}"
        
        inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(device)

        with torch.no_grad():
            # Kita atur parameter biar gak halusinasi (repetition_penalty)
            outputs = model.generate(
                **inputs, 
                max_length=128, 
                num_beams=4,
                repetition_penalty=1.5, # Hukuman biar gak ngulang kata
                early_stopping=True
            )
        
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Bersihkan hasil (buang link dll)
        pred_bersih = bersihkan_teks(pred)
        
        predictions.append(pred_bersih)
        references.append([ref_text])

    # Hitung Skor
    results = metric.compute(predictions=predictions, references=references)
    
    print("\n" + "="*30)
    print(f"🏆 REAL BLEU SCORE: {results['score']:.2f}")
    print("="*30)
    
    # Tampilkan 5 contoh hasil
    print("\n🔍 5 CONTOH HASIL:")
    for i in range(5):
        print(f"🇮🇩 Indo  : {df_sample.iloc[i][src_col]}")
        print(f"🤖 Model : {predictions[i]}")
        print(f"🔑 Kunci : {references[i][0]}")
        print("-" * 20)

if __name__ == "__main__":
    main()

📂 Memuat model...
📂 Membaca data tes: D:/UNAIR/NLP/Project_Madura/dataset4/test.csv
✅ Menguji pada 100 kalimat pertama.
🚀 Mulai Menerjemahkan...


100%|██████████| 100/100 [02:13<00:00,  1.34s/it]



🏆 REAL BLEU SCORE: 18.89

🔍 5 CONTOH HASIL:
🇮🇩 Indo  : Dekat dengan hotel saya menginap, hanya ditempuh jalan kaki, di sini banyak sekali pilihan makanannya, tempat yang luas, dan menyenangkan
🤖 Model : semma' sareng hotel kaula ngenep, coma e ajhâlân kaki, è dinna' bânnya' pilihan kakanan, kennengngan sè lebar, bân senneng
🔑 Kunci : Semmak bik hotel engkok nginep, pera' ejeleni ajelen soko, ediye bennyak sarah pelean kakananna, kenengngan se leber, ben masenneng
--------------------
🇮🇩 Indo  : Iya benar, dia sedang jaga warung.
🤖 Model : Iya bendher, rowa teppa' jaga warung.
🔑 Kunci : Iye bhender, rua ajege berung.
--------------------
🇮🇩 Indo  : Kangkungnya lumayan tapi kepiting saus padangnya mengecewakan kami dikasih kepiting yang kopong akhir kami tidak makan keptingnya dan dikembalikan.
🤖 Model : Kangkungna pendhenan tape kepiting saos padhangnga ngacewaaghi kami eberri' kepiting se kopong akhir kami ta' ngakan kepiting ban ekembaliaghi.
🔑 Kunci : Kangkongnga pendhanan tape kope

In [5]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
from tqdm import tqdm
import re

# ==========================================
# ⚙️ KONFIGURASI
# ==========================================
# 1. Path Model
MODEL_PATH = r"D:/UNAIR/NLP/Project_Madura/model_google_mt5"

# 2. Path File Test
FILE_TEST_PATH = r"D:/UNAIR/NLP/Project_Madura/dataset4/test.csv"

# ==========================================
# 🔧 FUNGSI BERSIH-BERSIH
# ==========================================
def bersihkan_teks(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    return text.strip()

# ==========================================
# 🚀 EVALUASI FULL (MADURA -> INDO)
# ==========================================
def main():
    print(f"📂 Memuat model...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
        model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(device)
    except:
        print("❌ Model tidak ketemu. Cek path lagi.")
        return

    # Load Data Test
    print(f"📂 Membaca data tes: {FILE_TEST_PATH}")
    try:
        df = pd.read_csv(FILE_TEST_PATH)
        
        # Deteksi nama kolom
        cols = [c.lower() for c in df.columns]
        col_indo, col_madura = None, None
        
        # Cari kolom Indo
        if 'indonesian' in cols: col_indo = 'indonesian'
        elif 'indonesia' in cols: col_indo = 'indonesia'
        
        # Cari kolom Madura
        if 'madurese' in cols: col_madura = 'madurese'
        elif 'madura' in cols: col_madura = 'madura'
        
        if not col_indo or not col_madura:
            print("❌ Kolom tidak lengkap.")
            return
            
        # Ambil 100 data pertama
        df_sample = df.head(400) 
        print(f"✅ Menguji arah MADURA -> INDONESIA pada {len(df_sample)} kalimat.")
        
    except Exception as e:
        print(f"❌ Gagal baca CSV: {e}")
        return

    metric = evaluate.load("sacrebleu")
    predictions = []
    references = []

    print("🚀 Mulai Menerjemahkan...")
    
    for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
        # --- PERUBAHAN UTAMA DI SINI ---
        
        # 1. Input sekarang Bahasa MADURA
        src_text = str(row[col_madura]) 
        
        # 2. Kunci Jawaban sekarang Bahasa INDONESIA
        ref_text = str(row[col_indo])   
        
        # 3. Prefix DIBALIK
        input_text = f"terjemahkan dari Bahasa Madura ke Bahasa Indonesia: {src_text}"
        # -------------------------------
        
        inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_length=128, 
                num_beams=4,
                repetition_penalty=1.5,
                early_stopping=True
            )
        
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred_bersih = bersihkan_teks(pred)
        
        predictions.append(pred_bersih)
        references.append([ref_text])

    # Hitung Skor
    results = metric.compute(predictions=predictions, references=references)
    
    print("\n" + "="*30)
    print(f"🏆 REAL BLEU SCORE (MAD -> INDO): {results['score']:.2f}")
    print("="*30)
    
    # Tampilkan 5 contoh hasil
    print("\n🔍 5 CONTOH HASIL:")
    for i in range(5):
        # Tampilkan label yang benar
        print(f"🐂 Madura : {df_sample.iloc[i][col_madura]}") 
        print(f"🤖 Model  : {predictions[i]}")
        print(f"🇮🇩 Kunci  : {references[i][0]}") # Kunci Jawaban Indo
        print("-" * 20)

if __name__ == "__main__":
    main()

📂 Memuat model...
📂 Membaca data tes: D:/UNAIR/NLP/Project_Madura/dataset4/test.csv
✅ Menguji arah MADURA -> INDONESIA pada 400 kalimat.
🚀 Mulai Menerjemahkan...


100%|██████████| 400/400 [12:28<00:00,  1.87s/it]



🏆 REAL BLEU SCORE (MAD -> INDO): 43.01

🔍 5 CONTOH HASIL:
🐂 Madura : Semmak bik hotel engkok nginep, pera' ejeleni ajelen soko, ediye bennyak sarah pelean kakananna, kenengngan se leber, ben masenneng
🤖 Model  : dekat dengan hotel saya menginap, hanya dijalankan jalan kaki, di sini banyak sekali pilihan makanannya, tempat yang luas, dan menyenangkan
🇮🇩 Kunci  : Dekat dengan hotel saya menginap, hanya ditempuh jalan kaki, di sini banyak sekali pilihan makanannya, tempat yang luas, dan menyenangkan
--------------------
🐂 Madura : Iye bhender, rua ajege berung.
🤖 Model  : iya benar, itu jaga warung.
🇮🇩 Kunci  : Iya benar, dia sedang jaga warung.
--------------------
🐂 Madura : Kangkongnga pendhanan tape kopeteng saos padangnga ma kocaba, engko' bi' laenna e bharri' kopeteng se kopong akherra engko' bi' laenna ta' ngakan kopeteng ban e pabali.
🤖 Model  : makasih lumayan tapi kopeteng saos padangnya mengatakan, saya dan lainnya dikasih kopeteng yang kopong akhirnya saya tidak makan kopeten

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re
import os

# ==========================================
# ⚙️ KONFIGURASI PATH
# ==========================================
MODEL_PATH = r"D:/UNAIR/NLP/Project_Madura/model_google_mt5"

# ==========================================
# 🔧 FUNGSI PEMBERSIH (Opsional)
# ==========================================
def bersihkan_hasil(text):
    # Hapus spasi ganda
    text = re.sub(r'\s+', ' ', text).strip()
    # (Opsional) Hapus tanda petik jika ingin hasil polos
    # text = re.sub(r"['`’‘]", "", text) 
    return text

# ==========================================
# 🚀 APLIKASI UTAMA
# ==========================================
def main():
    print("="*50)
    print("🤖 MEMUAT MODEL PENERJEMAH MADURA...")
    print(f"📂 Path: {MODEL_PATH}")
    print("="*50)

    # 1. Cek Device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"⚡ Sedang memuat ke {device.upper()}... Mohon tunggu.")

    # 2. Load Model
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
        model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(device)
        print("✅ Model SIAP digunakan!")
    except Exception as e:
        print(f"❌ Gagal load model: {e}")
        print("Pastikan path benar dan library protobuf sudah diinstall.")
        return

    # 3. Loop Interaktif
    while True:
        print("\n" + "-"*30)
        print("PILIH MENU:")
        print("1. Indo -> Madura")
        print("2. Madura -> Indo")
        print("3. Keluar")
        
        pilihan = input("👉 Masukkan angka (1/2/3): ").strip()
        
        if pilihan == '3':
            print("👋 Sampai jumpa!")
            break
        
        if pilihan not in ['1', '2']:
            print("⚠️ Pilihan salah, coba lagi.")
            continue

        # Minta Input Kalimat
        print("\n📝 Ketik kalimat yang ingin diterjemahkan:")
        kalimat = input("👉 Input: ").strip()
        
        if not kalimat:
            print("⚠️ Kalimat kosong.")
            continue

        # Tentukan Prefix
        if pilihan == '1':
            prefix = "terjemahkan dari Bahasa Indonesia ke Bahasa Madura: "
            arah = "🇮🇩 INDO -> 🐂 MADURA"
        else:
            prefix = "terjemahkan dari Bahasa Madura ke Bahasa Indonesia: "
            arah = "🐂 MADURA -> 🇮🇩 INDO"

        # Gabung Prefix + Kalimat
        input_text = prefix + kalimat
        
        # Proses Translate
        inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_length=128, 
                num_beams=5,            # Beam 5 biar mikir lebih cerdas
                repetition_penalty=1.2, # Cegah kata berulang
                early_stopping=True
            )
        
        hasil = tokenizer.decode(outputs[0], skip_special_tokens=True)
        hasil_bersih = bersihkan_hasil(hasil)

        # Tampilkan Hasil
        print("\n" + "="*40)
        print(f"🔄 {arah}")
        print(f"📝 Input : {kalimat}")
        print(f"🤖 Hasil : {hasil_bersih}")
        print("="*40)

if __name__ == "__main__":
    main()

🤖 MEMUAT MODEL PENERJEMAH MADURA...
📂 Path: D:/UNAIR/NLP/Project_Madura/model_google_mt5
⚡ Sedang memuat ke CUDA... Mohon tunggu.


c:\Users\MSI-PC\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


✅ Model SIAP digunakan!

------------------------------
PILIH MENU:
1. Indo -> Madura
2. Madura -> Indo
3. Keluar

📝 Ketik kalimat yang ingin diterjemahkan:

🔄 🇮🇩 INDO -> 🐂 MADURA
📝 Input : halo
🤖 Hasil : halo

------------------------------
PILIH MENU:
1. Indo -> Madura
2. Madura -> Indo
3. Keluar

📝 Ketik kalimat yang ingin diterjemahkan:

🔄 🇮🇩 INDO -> 🐂 MADURA
📝 Input : halo apa kabar
🤖 Hasil : halo apa kabar

------------------------------
PILIH MENU:
1. Indo -> Madura
2. Madura -> Indo
3. Keluar
👋 Sampai jumpa!


In [2]:
import torch
import pandas as pd
import re
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
from tqdm import tqdm

# ==========================================
# ⚙️ KONFIGURASI PATH (SESUAI REQUEST)
# ==========================================
# 1. Lokasi Model (Sesuai log error Bos sebelumnya)
MODEL_PATH = r"D:/UNAIR/NLP/Project_Madura/model_google_mt5"

# 2. Lokasi Dataset (Asumsi ada di folder dataset_nusax di drive D)
# Pastikan nama file csv-nya sesuai dengan yang ada di laptop Bos
PATH_NUSAX   = r"D:/UNAIR/NLP/Project_Madura/dataset4/test.csv"       # File Test NusaX
PATH_LEXICON = r"D:/UNAIR/NLP/Project_Madura/dataset4/madurese.csv"   # File Kamus (Lexicon)
PATH_INMAD   = r"D:/UNAIR/NLP/Project_Madura/dataset3/INMAD Dataset.csv" # File Formal INMAD

# Jumlah soal per tipe data (Total soal = 3 x 50 = 150 soal)
JUMLAH_SAMPEL_PER_DATA = 50 

# ==========================================
# 🧹 FUNGSI PEMBERSIH EJAAN (JURUS RAHASIA)
# ==========================================
def normalisasi_madura(text):
    text = str(text).lower()
    
    # 1. Buang Link & Mention sampah
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    
    # 2. HAPUS TANDA PETIK & EJAAN KHAS
    # Ini biar 'nase`' == 'nase' == "nase'"
    # Kita hapus petik satu ('), backtick (`), dan petik miring (’)
    text = re.sub(r"['`’‘]", "", text) 
    
    # 3. Samakan Ejaan Umum (Opsional)
    # Kadang 'dh' ditulis 'd', 'bh' ditulis 'b', 'jh' ditulis 'j'
    text = text.replace("dh", "d").replace("bh", "b").replace("jh", "j")
    
    # 4. Hapus tanda baca lain & spasi ganda
    text = re.sub(r"[^\w\s]", "", text)
    return text.strip()

# ==========================================
# 📂 LOAD DATA MIX (Chat + Kamus + Formal)
# ==========================================
def load_data_ujian():
    soal_ujian = []
    
    # A. NUSAX (Bahasa Kasar/Chat)
    if os.path.exists(PATH_NUSAX):
        try:
            df = pd.read_csv(PATH_NUSAX)
            cols = [c.lower() for c in df.columns]
            src, tgt = None, None
            if 'indonesian' in cols: src = 'indonesian'
            elif 'indonesia' in cols: src = 'indonesia'
            if 'madurese' in cols: tgt = 'madurese'
            elif 'madura' in cols: tgt = 'madura'
            
            if src and tgt:
                sample = df.sample(n=min(len(df), JUMLAH_SAMPEL_PER_DATA), random_state=42)
                for _, row in sample.iterrows():
                    soal_ujian.append({"tipe": "Sosmed (NusaX)", "indo": row[src], "madura": row[tgt]})
                print(f"✅ Masuk: {len(sample)} soal dari NusaX.")
        except: print("⚠️ Gagal baca NusaX.")
    else:
        print(f"❌ File tidak ketemu: {PATH_NUSAX}")

    # B. LEXICON (Kamus)
    if os.path.exists(PATH_LEXICON):
        try:
            df = pd.read_csv(PATH_LEXICON)
            sample = df.sample(n=min(len(df), JUMLAH_SAMPEL_PER_DATA), random_state=42)
            for _, row in sample.iterrows():
                soal_ujian.append({"tipe": "Kamus (Lexicon)", "indo": row['indonesian'], "madura": row['madurese']})
            print(f"✅ Masuk: {len(sample)} soal dari Lexicon.")
        except: print("⚠️ Gagal baca Lexicon.")
    else:
        print(f"❌ File tidak ketemu: {PATH_LEXICON}")

    # C. INMAD (Formal)
    if os.path.exists(PATH_INMAD):
        try:
            df = pd.read_csv(PATH_INMAD)
            df.columns = [c.strip() for c in df.columns]
            sample = df.sample(n=min(len(df), JUMLAH_SAMPEL_PER_DATA), random_state=42)
            for _, row in sample.iterrows():
                soal_ujian.append({"tipe": "Formal (INMAD)", "indo": row['Indonesia'], "madura": row['Madura']})
            print(f"✅ Masuk: {len(sample)} soal dari INMAD.")
        except: print("⚠️ Gagal baca INMAD.")
    else:
        print(f"❌ File tidak ketemu: {PATH_INMAD}")
    
    return pd.DataFrame(soal_ujian)

# ==========================================
# 🚀 MULAI PENILAIAN
# ==========================================
def main():
    print(f"\n🤖 MEMUAT MODEL DARI: {MODEL_PATH}")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"⚡ Device: {device.upper()}")
    
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
        model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(device)
    except Exception as e:
        print(f"❌ MODEL GAGAL DILOAD: {e}")
        print("Saran: Pastikan path foldernya benar dan sudah install protobuf.")
        return

    df_test = load_data_ujian()
    if len(df_test) == 0:
        print("❌ Tidak ada soal ujian! Cek path file csv mu.")
        return

    print(f"\n🚀 MULAI UJIAN ({len(df_test)} Soal)...")
    print("Parameter: Beam=5, Normalisasi=AKTIF (Anti-Petik)\n")
    
    metric = evaluate.load("sacrebleu")
    preds, refs = [], []
    
    # Loop Evaluasi
    for i, row in tqdm(df_test.iterrows(), total=len(df_test)):
        indo = str(row['indo'])
        kunci_raw = str(row['madura'])
        
        # Format Input (Wajib sama dengan training)
        text_input = f"terjemahkan dari Bahasa Indonesia ke Bahasa Madura: {indo}"
        inputs = tokenizer(text_input, return_tensors="pt", max_length=128, truncation=True).to(device)
        
        # Generate Jawaban
        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_length=128, 
                num_beams=5,             # Cari 5 kemungkinan terbaik
                repetition_penalty=1.2,  # Jangan ngulang kata
                early_stopping=True
            )
            
        jawaban_model = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # --- PROSES PEMBERSIHAN (NORMALISASI) ---
        jawaban_bersih = normalisasi_madura(jawaban_model)
        kunci_bersih   = normalisasi_madura(kunci_raw)
        # ----------------------------------------
        
        preds.append(jawaban_bersih)
        refs.append([kunci_bersih])
        
        # Tampilkan contoh setiap 25 soal
        if i % 25 == 0:
            print(f"\n[{row['tipe']}] Indo: {indo}")
            print(f"🤖 Jawab : {jawaban_model}  \t-> (Norm: {jawaban_bersih})")
            print(f"🔑 Kunci : {kunci_raw}      \t-> (Norm: {kunci_bersih})")

    # Hitung Skor Akhir
    score = metric.compute(predictions=preds, references=refs)
    
    print("\n" + "="*40)
    print(f"🏆 SKOR BLEU FINAL (ADIL): {score['score']:.2f}")
    print("="*40)
    
    if score['score'] > 20:
        print("✅ Kualitas BAGUS! (Model paham makna, walau beda gaya)")
    elif score['score'] > 10:
        print("⚠️ LUMAYAN. (Perlu perbaikan gaya bahasa)")
    else:
        print("❌ RENDAH. (Mungkin data training terlalu sedikit/kotor)")

if __name__ == "__main__":
    main()


🤖 MEMUAT MODEL DARI: D:/UNAIR/NLP/Project_Madura/model_google_mt5
⚡ Device: CUDA


c:\Users\MSI-PC\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


✅ Masuk: 50 soal dari NusaX.
✅ Masuk: 50 soal dari Lexicon.
✅ Masuk: 50 soal dari INMAD.

🚀 MULAI UJIAN (150 Soal)...
Parameter: Beam=5, Normalisasi=AKTIF (Anti-Petik)



  1%|          | 1/150 [00:04<11:36,  4.67s/it]


[Sosmed (NusaX)] Indo: Untuk menuju ke tempat ini jalannya sangat macet dan jauh dari pusat kota. Sesampainya di sana tidak ada hal yang spesial yang membuat perjalanan jauh terbayarkan. Menunya biasa saja.
🤖 Jawab : Kaangghuy entar ka kennengngan panèka jhâlânna cè' macet bân jhâu dâri pusat kottha. Sampè' è dhissa' tadâ' hal sè istimewa sè aghâbây parjhâlânan jhâu èbâyar. Menuna biasa sajan.  	-> (Norm: kaangghuy entar ka kennengngan panèka jâlânna cè macet bân jâu dâri pusat kottha sampè è dissa tadâ hal sè istimewa sè aghâbây parjâlânan jâu èbâyar menuna biasa sajan)
🔑 Kunci : Untuk depak ka kennengngan ria jhelenna macet ben jheu deri pusat kota. Sa depak en ka dissak tadek se spesial se ghebey perjhelenan jheu ria terbayarkan. Menuna biasa bhei.      	-> (Norm: untuk depak ka kennengngan ria jelenna macet ben jeu deri pusat kota sa depak en ka dissak tadek se spesial se ghebey perjelenan jeu ria terbayarkan menuna biasa bei)


...


[Formal (INMAD)] Indo: Kau yang aku tunggu dengan Afgan oleh rossa?
🤖 Jawab : Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi Budi [URL]  	-> (Norm: budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi budi url)
🔑 Kunci : edisi posang malem minggu ? Kamu yang kutunggu ( bik Afgan) deri Rossa ? http://path.com/p/nrLjm      	-> (Norm: edisi posang malem minggu  kamu yang kutunggu  bik afgan deri rossa)


 84%|████████▍ | 126/150 [02:54<00:36,  1.53s/it]


[Formal (INMAD)] Indo: dia memberiku handuk kotor tidak ada wifi di dalam ruangan jangan katakan wifi!
🤖 Jawab : kaulâh aberri' handuk gheddhe' tadâ' wifi è kamar jhâ' èkabâ'aghi wifi!  	-> (Norm: kaulâh aberri handuk ghedde tadâ wifi è kamar jâ èkabâaghi wifi)
🔑 Kunci : sempat eberri' andok gheddhe'. tade' WiFi e kamar. jhe' nyator bede WiFi!      	-> (Norm: sempat eberri andok ghedde tade wifi e kamar je nyator bede wifi)


100%|██████████| 150/150 [03:40<00:00,  1.47s/it]



🏆 SKOR BLEU FINAL (ADIL): 14.32
⚠️ LUMAYAN. (Perlu perbaikan gaya bahasa)
